In [0]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

In [0]:
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));
var defaultVarsBE = Workspace.Query<IfrsVariable>();

In [0]:
var yieldCurve_Test = @"
@@YieldCurve
Currency,Year,Month,Scenario,Values0,Values1,Values2,Values3
CHF,2021,6,,0.003,0.003,0.003,0.003
CHF,2021,6,MTUP10pct,0.0033,0.0033,0.0033,0.0033";

In [0]:
var cashflows_BestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2021,6,
@@Cashflow
DataNode,AmountType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,CL,C,,110,0,0,110,0,0,110,0,0,110,0,0,0,110,0,0,110,0,0,110,0,0,110,0
DT1.1,NIC,CL,C,,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5
DT1.1,CU,CL,C,,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-3.3
DT1.1,RA,CL,C,,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75";


In [0]:
var yieldCurveLog = await Import.FromString(yieldCurve_Test).WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync();
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));

In [0]:
Activity.Start();
    //var args = (await GetArgsFromMainAsync<PartitionByReportingNodeAndPeriod>(dataSet)) with {ImportFormat = ImportFormats.Cashflow};
    var args = new ImportArgs ("CH", 2021, 6, Periodicity.Quarterly, null, ImportFormats.Cashflow);
    await Import.FromString(cashflows_BestEstimate).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync();
    var calculatingPartitions = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().ToArrayAsync())
                                            .Where(x => x.ReportingNode == args.ReportingNode
                                                   && x.Year == args.Year
                                                   && x.Month == args.Month)
                                            .Select(x => new Guid (x.Id));
    foreach (var partition in calculatingPartitions)
    {
        continue;
    }
    /*var storage = new ImportStorage(args, DataSource, Workspace);
    await storage.InitializeAsync();
    if(Activity.HasErrors()) return Activity.Finish().Merge(parsingLog);
   
    var universe = Scopes.ForStorage(storage).ToScope<IModel>();
    var identities = universe.GetScopes<GetIdentities>(storage.DataNodesByImportScope[ImportScope.Primary]).SelectMany(s => s.Identities);
    var ivs = universe.GetScopes<ComputeAllScopes>(identities).SelectMany(x => x.CalculatedIfrsVariables);
    if(Activity.HasErrors()) return Activity.Finish().Merge(parsingLog);
    
    await Workspace.UpdateAsync<IfrsVariable>(ivs);
    await CommitToDatabase<IfrsVariable>(storage.TargetPartition, 
                                         snapshot : true, 
                                         filter : x => storage.EstimateTypesByImportFormat[ImportFormats.Cashflow].Contains(x.EstimateType) && 
                                                       storage.DataNodesByImportScope[ImportScope.Primary].Contains(x.DataNode));  
    await CommitToDatabase<RawVariable>(storage.TargetPartition, 
                                        snapshot : true,
                                        filter : x => storage.DataNodesByImportScope[ImportScope.Primary].Contains(x.DataNode));*/

    return Activity.Finish(); 

In [0]:
calculatingPartitions